In [21]:
%pip install pytesseract
%pip install pdf2image

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# Check if tesseract is installed
import os
os.system('which tesseract')

1

In [23]:
poppler_path = r'C:\Users\hp\Release-24.08.0-0\poppler-24.08.0\Library\bin'

In [24]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # adjust path if needed

In [25]:
import pdf2image
import pytesseract
import pandas as pd
import os
from pdf2image import convert_from_path
# Check if the PDF file exists
if not os.path.exists('testingpdf2.pdf'):
    print("PDF file not found!")
else:
    print("PDF file found, proceeding with extraction...")
    
    # Convert PDF pages to images
    try:
        images = convert_from_path('testingpdf2.pdf',poppler_path=poppler_path)
        print(f"Successfully converted {len(images)} pages to images")
        
        text_data = []
        # Process each image with pytesseract OCR to extract text
        for i, img in enumerate(images):
            print(f"Processing page {i+1}...")
            text = pytesseract.image_to_string(img)
            print(f"Text extracted from page {i+1}: {len(text)} characters")
            
            # Split the text into lines, ignoring empty lines
            lines = [line for line in text.split('\n') if line.strip()]

            for line in lines:
                text_data.append({'Page': i+1, 'Text': line.strip()})
        
        # Create a DataFrame and export to Excel
        df_text = pd.DataFrame(text_data)
        print(f"Created DataFrame with {len(df_text)} rows of text")
        
        # Display a sample of the extracted text
        if not df_text.empty:
            print("\
Sample of extracted text:")
            print(df_text.head())
            
            excel_filename = 'converted_pdf_text.xlsx'
            df_text.to_excel(excel_filename, index=False)
            print('PDF text extracted using OCR and saved as ' + excel_filename)
        else:
            print("No text was extracted from the PDF")
    except Exception as e:
        print(f"Error during processing: {str(e)}")

PDF file found, proceeding with extraction...
Successfully converted 18 pages to images
Processing page 1...
Text extracted from page 1: 2006 characters
Processing page 2...
Text extracted from page 2: 2958 characters
Processing page 3...
Text extracted from page 3: 2971 characters
Processing page 4...
Text extracted from page 4: 2608 characters
Processing page 5...
Text extracted from page 5: 3127 characters
Processing page 6...
Text extracted from page 6: 2748 characters
Processing page 7...
Text extracted from page 7: 2582 characters
Processing page 8...
Text extracted from page 8: 2954 characters
Processing page 9...
Text extracted from page 9: 2911 characters
Processing page 10...
Text extracted from page 10: 3362 characters
Processing page 11...
Text extracted from page 11: 2737 characters
Processing page 12...
Text extracted from page 12: 2812 characters
Processing page 13...
Text extracted from page 13: 2892 characters
Processing page 14...
Text extracted from page 14: 2630 cha

In [26]:
import pandas as pd

# Load the extracted text data
df = pd.read_excel('converted_pdf_text.xlsx')

# Display a sample to understand the structure
print("Sample of extracted text:")
print(df.head(20))

Sample of extracted text:
    Page                                      Text
0      1  Operating Account Consolidated Statement
1      1                                 Company :
2      1                          Account Number :
3      1                            Post Date From
4      1                              : 01/05/2018
5      1                                Currency :
6      1                 Transaction Amount From :
7      1                                 Report ID
8      1          S K S TEXTILES LIMITED (Company)
9      1              22506143895 (Account Number)
10     1                            Account Name :
11     1                                    Bank :
12     1                                Currency :
13     1                    Opening Ledger Balance
14     1                 Opening Available Balance
15     1                    Opening Balance As On:
16     1               Transaction Date Value Date
17     1                     Transaction Reference
18   

In [27]:
import re
import pandas as pd

# Read the OCR extracted excel file
df = pd.read_excel('converted_pdf_text.xlsx')

# Group the rows by page
pages = {}
for idx, row in df.iterrows():
    page = row['Page']
    text = row['Text']
    if page not in pages:
        pages[page] = []
    pages[page].append(text)

# We'll store headers and transactions separately
header_records = []
transaction_records = []

# Process each page
for page_num, lines in pages.items():
    header_info = {}
    transaction_columns = []
    transactions_started = False
    for line in lines:
        # Look for the transaction table header line
        if not transactions_started and re.search(r'Transaction Date', line, re.IGNORECASE):
            transactions_started = True
            # split by one or more whitespace
            transaction_columns = re.split(r'\s{2,}', line.strip())
            continue
        
        if transactions_started:
            # Likely transaction row
            # split using any whitespace and filter out empties
            parts = re.split(r'\s{2,}', line.strip())
            if len(parts) >= len(transaction_columns):
                # It may have extra parts or exactly the same count
                record = {'Page': page_num}
                for i, col in enumerate(transaction_columns):
                    if i < len(parts):
                        record[col] = parts[i]
                    else:
                        record[col] = ''
                transaction_records.append(record)
            else:
                # Combine with previous transaction if possible? Or skip
                # We'll assume this line doesn't belong to a transaction row
                pass
        else:
            # Header info parsing: if line contains ':' then separate key and value
            if ':' in line:
                parts = line.split(':', 1)
                key = parts[0].strip()
                value = parts[1].strip()
                header_info[key] = value
            else:
                # if line does not contain colon, may be title or subordinate info
                if 'Statement' in line or 'Account' in line or 'Bank' in line:
                    header_info.setdefault('Other', '')
                    header_info['Other'] += line + ' ' 
    # Append page header info if available
    if header_info:
        header_info['Page'] = page_num
        header_records.append(header_info)

# Convert header info and transaction records to DataFrames
df_header = pd.DataFrame(header_records)

df_transactions = pd.DataFrame(transaction_records)

# Write to Excel with multiple sheets
excel_filename = 'final_converted_data.xlsx'
with pd.ExcelWriter(excel_filename) as writer:
    df_header.to_excel(writer, sheet_name='Header', index=False)
    df_transactions.to_excel(writer, sheet_name='Transactions', index=False)

print('Formatted data extracted from PDF and saved as ' + excel_filename)
print('done')

Formatted data extracted from PDF and saved as final_converted_data.xlsx
done


In [28]:
import pandas as pd

# Load the formatted Excel file
with pd.ExcelFile('final_converted_data.xlsx') as xls:
    header_df = pd.read_excel(xls, 'Header')
    transactions_df = pd.read_excel(xls, 'Transactions')

# Display header information
print("Header Information:")
print(header_df.head())

# Display transaction data
print("\
Transaction Data:")
print(transactions_df.head())

Header Information:
                                               Other  Company  Account Number  \
0  Operating Account Consolidated Statement 22506...      NaN             NaN   
1          Operating Account Consolidated Statement       NaN             NaN   
2          Operating Account Consolidated Statement       NaN             NaN   
3          Operating Account Consolidated Statement       NaN             NaN   
4          Operating Account Consolidated Statement       NaN             NaN   

   Unnamed: 3  Currency  Transaction Amount From  Account Name  Bank  \
0  01/05/2018       NaN                      NaN           NaN   NaN   
1         NaN       NaN                      NaN           NaN   NaN   
2         NaN       NaN                      NaN           NaN   NaN   
3         NaN       NaN                      NaN           NaN   NaN   
4         NaN       NaN                      NaN           NaN   NaN   

   Opening Balance As On  Page  
0                    NaN   

In [29]:
# Let's look at more of the raw data to understand the structure better
import pandas as pd

# Read the original OCR data
df = pd.read_excel('converted_pdf_text.xlsx')

# Look at a larger sample from page 1 to understand the transaction structure
page1_data = df[df['Page'] == 1]
print("Page 1 data sample:")
print(page1_data.iloc[15:30])

Page 1 data sample:
    Page                                              Text
15     1                            Opening Balance As On:
16     1                       Transaction Date Value Date
17     1                             Transaction Reference
18     1                                       : QLDSO606R
19     1                        M/S S K S TEXTILES LIMITED
20     1                                       SCBLINBBXXX
21     1                                      Indian Rupee
22     1                                 1 -132,359,314.08
23     1                                    : 2,640,685.92
24     1                                       01.May.2018
25     1                                Customer Reference
26     1  02.May.2018 02.May.2018 SE36701805021948 7346959
27     1  02.May.2018 02.May.2018 SE36701805021947 7346950
28     1  02.May.2018 02.May.2018 IN3670180502X586 7349583
29     1  02.May.2018 02.May.2018 R122180055420248 7373577


In [30]:
# Let's look at more data to understand the transaction structure
import pandas as pd

# Read the original OCR data
df = pd.read_excel('converted_pdf_text.xlsx')

# Look at a larger sample from page 1 to understand the transaction structure
page1_data = df[df['Page'] == 1]
print("Page 1 data sample (continued):")
print(page1_data.iloc[30:45])

Page 1 data sample (continued):
    Page                                              Text
30     1  03.May.2018 03.May.2018 SE36701805032491 7730830
31     1                                 Processing Cheque
32     1                                     Branch Number
33     1                                               225
34     1                                               225
35     1                                               225
36     1                                               225
37     1                                               225
38     1                              Standard Chartered ~
39     1                         Post Date To : 31/05/2018
40     1                             Debit / Credit : Both
41     1                           Transaction Amount To :
42     1                Generated On : 07/06/2018 14:25:03
43     1                                              1225
44     1                                 : -130,305,520.57


In [31]:
# Let's try a different approach to better structure the data
import pandas as pd
import re

# Read the OCR extracted excel file
df = pd.read_excel('converted_pdf_text.xlsx')

# First, let's extract the header information
header_info = {}
transaction_start_idx = None

# Find where transactions start
for idx, row in df.iterrows():
    if 'Transaction Date Value Date Transaction Reference' in row['Text']:
        transaction_start_idx = idx
        break

# Extract header information
if transaction_start_idx:
    header_rows = df.iloc[:transaction_start_idx]
    for _, row in header_rows.iterrows():
        text = row['Text']
        if ':' in text:
            parts = text.split(':', 1)
            key = parts[0].strip()
            value = parts[1].strip() if len(parts) > 1 else ""
            header_info[key] = value

# Now let's process the transaction data
# First, identify the column headers
column_headers = None
for idx, row in df.iterrows():
    if 'Transaction Date Value Date Transaction Reference' in row['Text']:
        column_headers = row['Text']
        break

# Define a function to parse transaction rows
def parse_transaction_rows(df, start_idx):
    transactions = []
    
    # Get the rows that might contain transaction data
    potential_transaction_rows = df.iloc[start_idx+1:]
    
    # Define patterns for transaction data
    date_pattern = r'\d{2}\.May\.\d{4}'
    
    current_transaction = {}
    for _, row in potential_transaction_rows.iterrows():
        text = row['Text']
        
        # Check if this is a new transaction (starts with a date)
        if re.match(date_pattern, text):
            # If we have a current transaction, add it to our list
            if current_transaction:
                transactions.append(current_transaction)
                current_transaction = {}
            
            # Parse the transaction line
            parts = text.split()
            
            # Basic structure: Transaction Date, Value Date, Transaction Reference, Customer Reference, Branch Number
            if len(parts) >= 5:
                current_transaction['Transaction Date'] = parts[0]
                current_transaction['Value Date'] = parts[1]
                current_transaction['Transaction Reference'] = parts[2]
                current_transaction['Customer Reference'] = parts[3]
                current_transaction['Branch Number'] = parts[4]
        
        # Check for transaction amount (D for debit, C for credit)
        elif text.startswith('D ') or text.startswith('Cc '):
            amount_type = 'Debit' if text.startswith('D ') else 'Credit'
            amount = text[2:].strip()
            current_transaction['Amount Type'] = amount_type
            current_transaction['Amount'] = amount
        
        # Otherwise, it might be transaction details
        elif not re.match(r'\d{2}\.May\.\d{4}', text) and text and not ':' in text:
            current_transaction.setdefault('Transaction Details', '')
            current_transaction['Transaction Details'] += text + ' '
    
    # Add the last transaction if there is one
    if current_transaction:
        transactions.append(current_transaction)
    
    return transactions

# Parse transactions
transactions = parse_transaction_rows(df, transaction_start_idx)

# Convert to DataFrame
header_df = pd.DataFrame([header_info])
transactions_df = pd.DataFrame(transactions)

# Write to Excel with multiple sheets
excel_filename = 'bank_statement_formatted.xlsx'
with pd.ExcelWriter(excel_filename) as writer:
    header_df.to_excel(writer, sheet_name='Account Information', index=False)
    transactions_df.to_excel(writer, sheet_name='Transactions', index=False)

print(f"Formatted data saved to {excel_filename}")

# Display samples of the formatted data
print("\
Account Information:")
print(header_df.head())

print("\
Transactions:")
print(transactions_df.head())

Formatted data saved to bank_statement_formatted.xlsx
Account Information:
  Company Account Number               Currency Transaction Amount From  \
0                         4,094,479.43                                    

  Account Name Bank Opening Balance As On Post Date To Debit / Credit  \
0                                           31/05/2018           Both   

  Transaction Amount To         Generated On Closing Balance As On      Page  
0                        07/06/2018 14:25:03                        10 of 18  
Transactions:
  Transaction Date   Value Date Transaction Reference Customer Reference  \
0      23.May.2018  23.May.2018      IN3670180523W860            1852374   
1      23.May.2018  23.May.2018             S39403191             180523   
2      24.May.2018  24.May.2018      SE36701805241356            1917227   
3      24.May.2018  24.May.2018      SE36701805241357            1917228   
4      24.May.2018  24.May.2018      SE36701805241358            1917256   